In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Levenshtein import distance as lev_distance

In [2]:
df = pd.read_csv("/content/drive/MyDrive/42SEOUL/prob-0101.csv")

In [17]:
df.head()

,개봉일,제목,배급사,감독,출연진,장르,장르_묶음
0,2023-01-04,스위치,"롯데컬처웍스, (주)롯데엔터테인먼트",마대윤,"권상우, 오정세, 이민정, 박소이, 김준",기타,기타
1,2023-01-05,강남좀비,(주)와이드릴리즈,이수성,"지일주, 지연",드라마,드라마
2,2023-01-12,별 볼일 없는 인생,(주)이놀미디어,서동현,정가은,멜로/로맨스,로맨스
3,2023-01-12,10일간의 애인,(주)그노스,이영용,송민경,멜로/로맨스,로맨스
4,2023-01-18,교섭,플렉스엠엔터테인먼트,임순례,"황정민, 현빈, 강기영",드라마,드라마


# 1~2 영화 개수, 배급사 수, 감독 수, 출연진 수, 장르 수를 파악하고 감독, 출연진 등 중복 조건들에 대해 파악한다.

In [13]:
# 영화 개수 (행의 수)
movie_count = len(df)
print(movie_count)
# 배급사 수 (중복 제거)
distributor_count = df['배급사'].nunique()
print(distributor_count)
# 감독 수 (중복 제거)
director_count = df['감독'].nunique()
print(director_count)
# 출연진 수 (중복 제거, 쉼표로 구분된 출연진 분리)
actor_count = len(set(','.join(df['출연진']).split(',')))
print(actor_count)
# 장르 수 (중복 제거)
genre_count = df['장르'].nunique()
print(genre_count)

59
52
58
268
20


# 3. 본인이 추천하고 싶은 기준을 설정한다. (장르)

In [15]:
# 장르 유형 파악
df['장르'].unique()

array(['기타', '드라마', '멜로/로맨스', '액션', 'SF', '멜로·로맨스·코미디', '스릴러', '범죄, 드라마',
       '드라마, 멜로·로맨스, 가족', '멜로·로맨스', '코미디, 액션', '다큐멘터리', '멜로,로맨스,드라마',
       '범죄·액션·스럴리', '드라마,멜로', '코미디', '공포·미스터리', '범죄, 액션', '공포,코미디', '범죄'],
      dtype=object)

In [16]:
# 장르_묶음 정의
genre_groups = {
    '로맨스': ['멜로/로맨스', '멜로·로맨스·코미디', '멜로·로맨스', '멜로,로맨스,드라마', '드라마,멜로'],
    '드라마': ['드라마', '범죄, 드라마'],
    '액션': ['액션', '코미디, 액션', '범죄·액션·스럴리', '범죄, 액션'],
    '코미디': ['코미디', '멜로·로맨스·코미디', '공포,코미디'],
    '공포/스릴러': ['스릴러', '공포·미스터리'],
    '범죄': ['범죄, 드라마', '범죄·액션·스럴리', '범죄, 액션', '범죄'],
    '다큐멘터리': ['다큐멘터리'],
    '기타': ['기타', 'SF']
}

In [ ]:
# 장르를 묶음으로 매핑
def map_genre_to_group(genre):
    for group, genres in genre_groups.items():
        if any(g in genre for g in genres):
            return group
    return '기타'

# 새 컬럼 생성
df['장르_묶음'] = df['장르'].apply(map_genre_to_group)

# 4. 본인의 기준에 의해 영화 몇개를 추천하는 것이 적당한지 실험해 본다.(장르, 3개)

In [24]:
df['장르_묶음'].unique()

array(['기타', '드라마', '로맨스', '액션', '공포/스릴러', '다큐멘터리', '코미디', '범죄'],
      dtype=object)

In [ ]:
def recommend_movies(df, genre, n=3):
    if genre not in df['장르_묶음'].unique():
        return "다시 입력하세요."
    recommended = df[df['장르_묶음'] == genre].sample(n=min(n, len(df[df['장르_묶음'] == genre])))
    return recommended[['제목', '장르_묶음']]

In [23]:
genre = input("추천받고 싶은 영화 장르를 입력하세요: ")
result = recommend_movies(df, genre)
result

추천받고 싶은 영화 장르를 입력하세요: 액션


,제목,장르_묶음
48,범죄도시 3,액션
54,귀공자,액션
33,나는 여기에 있다,액션


# 4-1 오타 방지 유사도 기반 장르 검색

In [37]:
genre_clusters = df['장르_묶음'].unique().tolist()

# TF-IDF 벡터라이저 초기화 및 적용
tfidf_vectorizer = TfidfVectorizer().fit(genre_clusters)
tfidf_matrix = tfidf_vectorizer.transform(genre_clusters)

def compute_cosine_similarity(user_input):
    user_input_vec = tfidf_vectorizer.transform([user_input])
    cosine_similarities = cosine_similarity(user_input_vec, tfidf_matrix).flatten()
    return cosine_similarities

def compute_levenshtein_similarity(user_input, genres):
    distances = [lev_distance(user_input.lower(), genre.lower()) for genre in genres]
    return [1 - (dist / max(len(user_input), len(genre))) for dist, genre in zip(distances, genres)]

def identify_closest_genre(user_input, cosine_threshold=0.1, levenshtein_threshold=0.6):
    cosine_similarities = compute_cosine_similarity(user_input)
    levenshtein_similarities = compute_levenshtein_similarity(user_input, genre_clusters)

    combined_similarities = [max(cos, lev) for cos, lev in zip(cosine_similarities, levenshtein_similarities)]

    best_match_index = np.argmax(combined_similarities)
    best_match_similarity = combined_similarities[best_match_index]

    if best_match_similarity >= max(cosine_threshold, levenshtein_threshold):
        return genre_clusters[best_match_index]
    else:
        return None

In [ ]:
def recommend_movies_by_genre(df, user_input, n=3):
    closest_genre = identify_closest_genre(user_input)

    if closest_genre is None:
        return f"입력한 장르 '{user_input}'와 유사한 장르를 찾을 수 없습니다. 다시 입력해주세요."

    recommended_movies = df[df['장르_묶음'] == closest_genre].sample(n=min(n, len(df[df['장르_묶음'] == closest_genre])))
    return closest_genre, recommended_movies[['제목', '장르_묶음']]

In [36]:
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.5 MB/s eta 0:00:00


In [38]:
while True:
    user_input = input("추천받고 싶은 영화 장르를 입력하세요 (종료하려면 'q' 입력): ")
    if user_input.lower() == 'q':
        break

    result = recommend_movies_by_genre(df, user_input)
    if isinstance(result, str):
        print(result)
    else:
        closest_genre, recommended_movies = result
        print(f"입력한 장르 '{user_input}'와 가장 유사한 장르: {closest_genre}")
        print("추천 영화:")
        print(recommended_movies)
    print()

추천받고 싶은 영화 장르를 입력하세요 (종료하려면 'q' 입력): 루맨스
입력한 장르 '루맨스'와 가장 유사한 장르: 로맨스
추천 영화:
                      제목 장르_묶음
29                솔라 플라워   로맨스
35  밥만 잘 사주는 이상한 이사님 극장판   로맨스
10       우리 사랑이 향기로 남을 때   로맨스

추천받고 싶은 영화 장르를 입력하세요 (종료하려면 'q' 입력): 엑숀
입력한 장르 '엑숀'와 유사한 장르를 찾을 수 없습니다. 다시 입력해주세요.

추천받고 싶은 영화 장르를 입력하세요 (종료하려면 'q' 입력): 액션
입력한 장르 '액션'와 가장 유사한 장르: 액션
추천 영화:
      제목 장르_묶음
5     유령    액션
22   웅남이    액션
42  바람개비    액션

추천받고 싶은 영화 장르를 입력하세요 (종료하려면 'q' 입력): 엑션
입력한 장르 '엑션'와 유사한 장르를 찾을 수 없습니다. 다시 입력해주세요.

추천받고 싶은 영화 장르를 입력하세요 (종료하려면 'q' 입력): 액숀
입력한 장르 '액숀'와 유사한 장르를 찾을 수 없습니다. 다시 입력해주세요.

추천받고 싶은 영화 장르를 입력하세요 (종료하려면 'q' 입력): 드라머
입력한 장르 '드라머'와 가장 유사한 장르: 드라마
추천 영화:
         제목 장르_묶음
40     2퍼센트   드라마
34     물안에서   드라마
30  사랑의 고고학   드라마

추천받고 싶은 영화 장르를 입력하세요 (종료하려면 'q' 입력): 드라머
입력한 장르 '드라머'와 가장 유사한 장르: 드라마
추천 영화:
          제목 장르_묶음
51     안나푸르나   드라마
27  오늘 출가합니다   드라마
40      2퍼센트   드라마

추천받고 싶은 영화 장르를 입력하세요 (종료하려면 'q' 입력): q


# 5. 다양한 접근 조회 방법을 실험한다. (감독)

In [41]:
sorted_df = df[['감독', '장르_묶음']].sort_values(by='감독')

print(sorted_df)

      감독   장르_묶음
44   [1]     코미디
47   가성문     드라마
13   곽정덕      액션
12   권혁재     드라마
14   김덕중     드라마
27   김성환     드라마
15   김주환     드라마
11   김태준  공포/스릴러
24   김현정     드라마
50   김홍기     코미디
0    마대윤      기타
40   문신구     드라마
18   민용근     드라마
19   박동기     드라마
22   박성광      액션
54   박훈정      액션
21   배인우     로맨스
2    서동현     로맨스
33   신근호      액션
56   신재호      액션
35   양경희     로맨스
38   양윤모     드라마
6    연상호      기타
57   윤여창      범죄
39   이병헌     드라마
48   이상용      액션
42   이상훈      액션
26   이소현   다큐멘터리
32  이송희일     드라마
1    이수성     드라마
3    이영용     로맨스
30   이완민     드라마
36   이원석     코미디
16   이원태     드라마
49   이종필     드라마
17   이창열     드라마
5    이해영      액션
45   이현준     로맨스
52    이황     코미디
10   임성용     로맨스
23   임성운     드라마
4    임순례     드라마
41   임재완     로맨스
25   장항준     드라마
43   장형모  공포/스릴러
8    정주리     드라마
37   조은성     로맨스
7     주영     드라마
31   최세환     드라마
46   최승연      기타
53   최윤호     드라마
55   최윤호     코미디
28   최종태     드라마
58   최주연  공포/스릴러
20   최창환     로맨스
29   한경탁     로맨스
9    형슬우     드라마
34   홍상수     드

In [54]:
# 감독별 빈도수 계산
director_counts = df['감독'].value_counts()

print(director_counts.head())

감독
최윤호     2
마대윤     1
장형모     1
최세환     1
이송희일    1
Name: count, dtype: int64


In [45]:
def recommend_movies(df, director, n=3):
    if director not in df['감독'].unique():
        return "다시 입력하세요."
    recommended = df[df['감독'] == director].sample(n=min(n, len(df[df['감독'] == director])))
    return recommended[['제목', '감독']]

In [50]:
director = input("추천받고 싶은 영화감독을 입력하세요: ")
result = recommend_movies(df, director)
result

추천받고 싶은 영화감독을 입력하세요: 최윤호


,제목,감독
53,실버맨,최윤호
55,손,최윤호


# 5. 다양한 접근 조회 방법을 실험한다. (출연진)

In [51]:
def count_cast(cast_string):
    if pd.isna(cast_string):
        return 0
    return len(cast_string.split(','))

def recommend_movies_by_cast_count(df, target_count, n=3):
    df['출연진_수'] = df['출연진'].apply(count_cast)
    df['차이'] = abs(df['출연진_수'] - target_count)
    recommended = df.sort_values('차이').head(n)

    return recommended[['제목', '출연진', '출연진_수']]

In [53]:
target_count = int(input("원하는 출연진 인원수를 입력하세요: "))
result = recommend_movies_by_cast_count(df, target_count)
print(result)

원하는 출연진 인원수를 입력하세요: 5
     제목                      출연진  출연진_수
0   스위치   권상우, 오정세, 이민정, 박소이, 김준      5
56  인드림  서효림, 오지호, 김승수, 이설구, 노지유      5
55    손   이재원, 박상욱, 정서하, 허웅, 안수호      5
